# Baxter Inverse Kinematics Introduction

Learn how to move the baxter in Cartesian space (inverse kinematics).

##### ENGG 4460 University of Guelph, Patrick Wspanialy

Import python modules needed to control Baxter

In [ ]:
import pprint
import math
import rospy
import baxter_interface
from baxter_interface import (RobotEnable, Gripper)
from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from std_msgs.msg import Header
from baxter_core_msgs.srv import (
    SolvePositionIK,
    SolvePositionIKRequest,
)
pp = pprint.PrettyPrinter(indent=4)

Initialize a new ROS node and connect to the inverse kinematics services for the right and left arms. 

In [ ]:
rospy.init_node("rsdk_ik_service_client")

right_namespace = "ExternalTools/right/PositionKinematicsNode/IKService"
left_namespace = "ExternalTools/left/PositionKinematicsNode/IKService"

right_ik_service = rospy.ServiceProxy(right_namespace, SolvePositionIK)
left_ik_service = rospy.ServiceProxy(left_namespace, SolvePositionIK)

right_ik_request = SolvePositionIKRequest()
left_ik_request = SolvePositionIKRequest()

ik_header = Header(stamp=rospy.Time.now(), frame_id='base')

Enable Baxter

In [ ]:
baxter = RobotEnable()
baxter.enable()

Choose the pose we want the right arm to go to. 

#### Position
- x: in, out. 0 is at the center of Baxter
- y: left, right. 0 is at the center of Baxter
- z: up, down. 0 is at the table surface (don't assign z to be less than 0)

#### Orientation
https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation


| x  | y  | z  |  w | description  |
|---|---|---|---|---|
| 0  | 0  | 0  | 1  | neutral position, no rotations applied  |
| 1  | 0  | 0  | 0  | 180° turn around X axis |
| 0  | 1  | 0  | 0  | 180° turn around Y axis  |
| 0  | 0  | 1  | 0  | 180° turn around Z axis  |
| math.sqrt(0.5)  | 0  | 0  | math.sqrt(0.5)  | 90° rotation around X axis  |
| 0  | math.sqrt(0.5)  | 0 | math.sqrt(0.5)  | 90° rotation around Y axis  |
| 0  | 0  | math.sqrt(0.5)  | math.sqrt(0.5)  | 90° rotation around Z axis  |
| -math.sqrt(0.5)  | 0  | 0  | math.sqrt(0.5)  | -90° rotation around X axis  |
| 0  | -math.sqrt(0.5)  | 0  | math.sqrt(0.5)  | -90° rotation around Y axis  |
| 0  | 0  |  -math.sqrt(0.5) | math.sqrt(0.5)  | -90° rotation around Z axis  |



In [ ]:
right_pose = PoseStamped(
            header=ik_header,
            pose=Pose(
                position=Point(
                    x=0.5,
                    y=-0.5,
                    z=0.6,
                ),
                orientation=Quaternion(
                    x=0.0,
                    y=0.0,
                    z=0.0,
                    w=1.0,
                ),
            ),
        )

Store our pose into our request variable.

In [ ]:
right_ik_request.pose_stamp[:] = []
right_ik_request.pose_stamp.append(right_pose)

print right_ik_request.pose_stamp

Try to get a joint solution to the pose we defined using the inverse kinematics engine running in Baxter.

In [ ]:
try:
    rospy.wait_for_service(right_namespace, 5.0)
    right_ik_response = right_ik_service(right_ik_request)
    if (right_ik_response.isValid[0]):
        right_limb_joints = dict(zip(right_ik_response.joints[0].name, right_ik_response.joints[0].position))
        pp.pprint(right_limb_joints)
    else:
        print("INVALID POSE - No Valid Joint Solution Found.")
except (rospy.ServiceException, rospy.ROSException), e:
    rospy.logerr("Service call failed: %s" % (e,))

If a joint solution to our pose exists, we can move the the joints positions the IK engine generated.

In [ ]:
right_arm = baxter_interface.Limb('right')
right_arm.move_to_joint_positions(right_limb_joints)

Let's print out the joint positions, along with the Cartesian position and Quaternion orientation

In [ ]:
right_angles = right_arm.joint_angles()
right_pose = right_arm.endpoint_pose()
pp.pprint(right_angles)
print "\n"
pp.pprint(right_pose['position'])
pp.pprint(right_pose['orientation'])
